#### Add dependencias

In [5]:
%pip install pandas
%pip install requests
%pip install python-dotenv
sdw2023_api_url = "https://sdw-2023-prd.up.railway.app"

from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


True

### Extract


In [6]:
# leitura dos dados e obtenção dos IDs
import pandas as pd

df = pd.read_csv("sdw2023.csv")
user_ids = df["UserID"].to_list()

In [7]:
import requests
import json

def get_user(id):
    response = requests.get(f"{sdw2023_api_url}/users/{id}")
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]

### Transform

In [8]:
%pip install openai
import os

openai_api_key = os.environ["openai_api_key"]

Note: you may need to restart the kernel to use updated packages.


In [11]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [
            {
                "role": "system",
                "content": "Você é um especialista em marketing bancário."
            },
            {
                "role": "user",
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 250 caracteres)"
            }
        ]
    )
    
    return completion.choices[0].message.content.strip('\"')

for user in users:
    news = generate_ai_news(user)
    print(news)
    user['news'].append({
        "description": news
    })


Maria, investir é essencial para alcançar seus objetivos financeiros. Seja para aposentadoria, uma casa nova ou educação dos filhos, invista com sabedoria e transforme seus sonhos em realidade. Conte com nosso apoio para guiar suas escolhas financeiras.
Roberta, invista no seu futuro! Os investimentos são fundamentais para garantir sua estabilidade financeira e realizar seus sonhos. Não deixe seu dinheiro parado, faça ele trabalhar para você.
Carla, investir é crucial para alcançar seus objetivos financeiros. Aproveite as oportunidades do mercado e garanta um futuro mais próspero e seguro. Conte com o nosso apoio para orientá-la nessa jornada. #InvestirParaCrescer


### Load

In [10]:
def update_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}")

User Maria updated? True
User Roberta updated? True
User Carla updated? True
